In [ ]:
# python 3.11.9
# установлены библиотеки без указания версий и это работает
# %pip install python-dotenv nest_asyncio xmltodict faiss-cpu langchain openai tiktoken langchain-openai
# %pip install uvicorn
# %pip install fastapi


## Пример использования простого API

In [2]:
import requests
from IPython.display import display, HTML
# from IPython.display import display_html, clear_output
import json

# запросы в fastapi_example
response = requests.get("http://127.0.0.1:5000/")
print(response.text)

{"message":"!answer!"}


In [3]:
response = requests.get("http://127.0.0.1:5000/users/85")
print(response.text)


{"user_id":"85"}


In [4]:
# запросы в fastapi_example
payload={"name":"Ivan", "description":"good item", "price":42.3}
response = requests.post("http://127.0.0.1:5000/users", json=payload)
print(response.text)

{"user_name":"Ivan","description":"good item","price":42.3}


In [5]:
response = requests.get("http://127.0.0.1:5000/html")
display(HTML(response.text))

In [6]:
response = requests.get("http://127.0.0.1:5000/calls")
data = json.loads(response.text)
display(HTML(f"количество вызовов каждой функции сервиса:</br>{data['message']}"))

## Пример использования модуля кастомного chatGPT 

In [8]:
# test model class without fastapi
from model import LLMModel
model = LLMModel("")
q1="какая деятельность относится к аэропортовой"
display(HTML(model.get_answer(q1)))


## Пример использования API с кастомным chatGPT 

In [10]:
# проверка логики модели - если не инициализирована, то выдать предупреждение:
payload = {"text":"какая деятельность относится к аэропортовой?"}
response = requests.post("http://127.0.0.1:5000/api/get_answer", json=payload)
print(response.text)

{"message":"model is not active. start by /api/start"}


In [64]:
# запуск модели
response = requests.get("http://127.0.0.1:5000/api/start")
print(response.text)

{"message":"model started and ready to answer"}


In [65]:
# использование кастомного chatgpt через кастомный api
payload = {"text":"какая деятельность относится к аэропортовой?"}
response = requests.post("http://127.0.0.1:5000/api/get_answer", json=payload)
data = json.loads(response.text)
display(HTML(f"Ответ ассистента:</br>{data['message']}"))

In [66]:
payload = {"text":"как исчисляется страховая премия?"}
response = requests.post("http://127.0.0.1:5000/api/get_answer", json=payload)
data = json.loads(response.text)
display(HTML(f"Ответ ассистента:</br>{data['message']}"))

In [67]:
response = requests.get("http://127.0.0.1:5000/calls")
data = json.loads(response.text)
display(HTML(f"количество вызовов каждой функции сервиса:</br>{data['message']}"))

# ^ задание выполнено

#  промежуточные тесты для платформы windows из-за проблем с SSL.
# в итоге ничего не получилось. итоговый код выше работает только для ubuntu.

In [ ]:
# python 3.11.4
# %pip install pygments pexpect
# python -m pip install python-certifi-win32
# python -m pip install certifi
#### %pip install certifi python-dotenv nest_asyncio faiss-cpu==1.7.4 langchain==0.1.7 openai==1.12.0 tiktoken==0.6.0 langchain_community==0.0.20 langchain-openai==0.0.6

# %pip install -r requirements.txt
# %pip install --upgrade pip
# %pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# %pip install -U langchain-openai
# установка openssl
# Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://chocolatey.org/install.ps1'))
# choco install openssl
%pip install --upgrade httpx


In [ ]:
import certifi
certifi.where()

'/home/userdisk/anaconda3/envs/env3119/lib/python3.11/site-packages/certifi/cacert.pem'

In [ ]:
%pip show langchain-openai
%pip show langchain
# Deprecated
# from langchain.embeddings.openai import OpenAIEmbeddings
# print(dir(OpenAIEmbeddings))
# print(OpenAIEmbeddings.__doc__)

In [ ]:
import runpy
import trace
# Создаем объект трассировки
tracer = trace.Trace(trace=True)
# Запускаем скрипт под трассировкой
tracer.run('runpy.run_path("model0.py")')

In [ ]:
# попытка подключить файл с гуглдрайва не из временной папки, а из постоянной
# не получилось

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
import httplib2
import google_auth_httplib2
import io
# from googleapiclient.discovery import build
import requests
from google.auth.transport.requests import Request
from google.auth.transport.requests import AuthorizedSession
import logging
logging.basicConfig(level=logging.DEBUG)

# найти на гуглдрайве файл, имя которого включает в себя uniquesubstringfromfilename и вернуть текст этого файла
# для получения сервис файла json надо настроить авторизацию гугл на google cloud
def getfilefromgoogledisk(uniquesubstringfromfilename=""):
    # Путь к файлу с учетными данными
    # "Копия ПРАВИЛА СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ"
    
    SERVICE_ACCOUNT_FILE = 'googledriveutopian-theater-338909-30c6f2cd39b6.json'

    # Создание сервисного аккаунта
    credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    h = httplib2.Http(".cache", disable_ssl_certificate_validation=True)
    authorized_http = google_auth_httplib2.AuthorizedHttp(credentials=credentials,http=h)
    
    # Создание сервиса Google Drive с использованием настроенного экземпляра AuthorizedHttp
    service = build('drive', 'v3', http=authorized_http)
    # Пример функции для получения списка файлов
    def list_files(keyword):
        try:
            results = service.files().list(
                pageSize=10, fields="nextPageToken, files(id, name)").execute()
            items = results.get('files', [])
            if not items:
                print('No files found.')
            else:
                print('Files:')
                for item in items:
                    if keyword in item['name']:
                        print(u'{0} ({1})'.format(item['name'], item['id']))
                        return item['id']
        except HttpError as error:
            print(f'An error occurred: {error}')
            items = None

    file_id=list_files(uniquesubstringfromfilename)

    # Получение метаданных файла для определения его типа
    file_metadata = service.files().get(fileId=file_id).execute()
    file_mime_type = file_metadata.get('mimeType')
    mime_type = 'text/plain' # Меняем MIME-тип на 'text/plain' для экспорта текста

    # Получение метаданных файла
    file_metadata = service.files().get(fileId=file_id).execute()
    file_mime_type = file_metadata.get('mimeType')

    # Определение, нужно ли использовать метод export
    if file_mime_type in ['application/vnd.google-apps.document', 'application/vnd.google-apps.spreadsheet', 'application/vnd.google-apps.presentation']:
        # Использование метода export для документов Google Docs, Sheets и Slides
        request = service.files().export_media(fileId=file_id, mimeType=mime_type)
    else:
        # Для других типов файлов используем метод get_media
        request = service.files().get_media(fileId=file_id)

    # Создание буфера для хранения текста
    text_content = io.BytesIO()

    # Загрузка текста в буфер
    downloader = MediaIoBaseDownload(text_content, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Загружено {}%.".format(int(status.progress() * 100)))

    # Извлечение текста из буфера
    text_content.seek(0) # Перемещение указателя в начало буфера
    text = text_content.read().decode('utf-8') # Чтение и декодирование текста

    return text
print(getfilefromgoogledisk("АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ")[:1000]) # Вывод первых 1000 символов текста


In [26]:
import functools
# декоратор для подсчета вызовов каждой функции
class CountCalls:
    calls = {} # Глобальный словарь для отслеживания количества вызовов
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        print(self.func)
    def __call__(self, *args, **kwargs):
        print(f"Calling {self.func.__name__}")
        func_name = self.func.__name__
        if func_name not in CountCalls.calls:
            CountCalls.calls[func_name] = 0
        CountCalls.calls[func_name] += 1
        return self.func(*args, **kwargs)
@CountCalls
def read_root():
    print(CountCalls.calls)
    return {"message": f"{CountCalls.calls}!!answer!"}
def get_calls():
    test = '\n'.join([f"{func}={cnt}" for func, cnt in CountCalls.calls.items()])
    return {"message": test}
@CountCalls
def users(id):
    return {"user_id": id}

<function read_root at 0x7bbfb84ff1a0>
<function users at 0x7bbfb84fe660>


In [29]:
users(11)
read_root()

Calling users
Calling read_root
{'users': 2, 'read_root': 2}


{'message': "{'users': 2, 'read_root': 2}!!answer!"}

In [30]:
get_calls()

{'message': 'users=2\nread_root=2'}